In [7]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms, models
from torchvision.io import decode_image
from sklearn.model_selection import KFold
import numpy as np
import os
import cv2
from PIL import Image

In [8]:
# Prepare the data - create a KFold split
random_seed = 42
np.random.seed(random_seed)
torch.manual_seed(random_seed)

# "CelebA" dataset
authentic_path = "dane/CelebA/authentic"
spoof_path = "dane/CelebA/spoof"

# Load images
authentic_images = [os.path.join(authentic_path, img) for img in os.listdir(authentic_path)]
spoof_images = [os.path.join(spoof_path, img) for img in os.listdir(spoof_path)]

# Create X and y from all images
X = authentic_images + spoof_images
y = np.concatenate((np.zeros(len(authentic_images)), np.ones(len(spoof_images)))).astype(np.int64)

# KFold
kf = KFold(n_splits=5, random_state=random_seed, shuffle=True)

In [9]:
# Custom dataset to load images and labels
class OurDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label, image_path  # Return image, label, and file path

In [10]:
# Create a text file to store incorrect predictions
with open("incorrect_predictions_convnext.txt", "w") as f:
    f.write("name\tpredicted\tactual\n")
    f.write("(0 - authentic, 1 - spoof)\n")

In [11]:
# Create a transform to resize images
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

In [12]:
# Create, test and evaluate the model for each fold
for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    print(f"Fold {fold}")
    X_train = [X[i] for i in train_idx]
    y_train = y[train_idx]
    X_test = [X[i] for i in test_idx]
    y_test = y[test_idx]
    
    # Create custom datasets
    train_dataset = OurDataset(X_train, y_train, transform=transform)
    test_dataset = OurDataset(X_test, y_test, transform=transform)
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    # Device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Create a model
    model = models.convnext_small(weights=None)
    num_ftrs = model.classifier[-1].in_features
    model.classifier[-1] = nn.Linear(num_ftrs, 2)  # Two output neurons
    model = model.to(device)

    # Train the model
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 10
    
    model.train()

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels, _ in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")
        
    # Evaluate the model
    correct = 0
    total = 0
    incorrect_files = []
    
    accuracy = 0

    model.eval()

    with torch.no_grad():
        for images, labels, file_paths in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Collect file names of incorrectly predicted images
            with open("incorrect_predictions_convnext.txt", "a") as f:
                for i in range(len(predicted)):
                    if predicted[i].item() != labels[i].item():
                        f.write(file_paths[i] + "\t" + str(predicted[i].item()) + "\t" + str(labels[i].item()) + "\n")

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    print("-" * 50)

Fold 0
Epoch 1/10, Loss: 0.7148364968275074
Epoch 2/10, Loss: 0.6693683343082408
Epoch 3/10, Loss: 0.48602789260795665
Epoch 4/10, Loss: 0.3371417495959553
Epoch 5/10, Loss: 0.3026034590663902
Epoch 6/10, Loss: 0.2653352783692668
Epoch 7/10, Loss: 0.24552736077877327
Epoch 8/10, Loss: 0.22573882557994054
Epoch 9/10, Loss: 0.20688605014770656
Epoch 10/10, Loss: 0.18881066821747988
Test Accuracy: 91.26%
--------------------------------------------------
Fold 1
Epoch 1/10, Loss: 0.7213006023724323
Epoch 2/10, Loss: 0.40454974109419917
Epoch 3/10, Loss: 0.3459707128733035
Epoch 4/10, Loss: 0.3253415241463036
Epoch 5/10, Loss: 0.3047782786358374
Epoch 6/10, Loss: 0.2839510084696411
Epoch 7/10, Loss: 0.26315677896839285
Epoch 8/10, Loss: 0.2387717620016074
Epoch 9/10, Loss: 0.22920187874543813
Epoch 10/10, Loss: 0.21034280041578118
Test Accuracy: 90.28%
--------------------------------------------------
Fold 2
Epoch 1/10, Loss: 0.570699611417239
Epoch 2/10, Loss: 0.35994145062395355
Epoch 3/